## PI-3 Running SFINCS using the National Water Model 3.0 inputs

Below is a notebook for running SFINCS using National Water Model inputs over a domain of interest

Author: Tadd Bindas

### Step 0: Import dependencies and create national water model data